# Project: Summarization App Using LangChain and OpenAI
This is part of "Learn LangChain, Pinecone & OpenAI: Build Next-Gen LLM Apps" course.

https://www.udemy.com/course/master-langchain-pinecone-openai-build-llm-applications/?referralCode=4B17E3BD4CBBEA3B8321

This notebook uses the latest versions of the OpenAI and LangChain libraries.

In [1]:
pip install -q -r ./requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## Basic Prompt

In [3]:
from langchain_openai import ChatOpenAI
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [4]:
text= r"""
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

messages = [
    SystemMessage(content='You are an expert copywriter with expertize in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text:\n TEXT: {text}')
]

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')


In [5]:
llm.get_num_tokens(text)

238

In [6]:
summary_output = llm.invoke(messages)

In [7]:
print(summary_output.content)

Mojo is a new programming language that combines Python's usability with C's performance, enabling enhanced programmability of AI hardware and extensibility of AI models. It bridges the gap between research and production by offering faster and portable code that seamlessly integrates with the Python ecosystem. Developed by Modular to simplify ML/AI infrastructure, Mojo recognizes the importance of host CPUs as accelerators and aims to streamline programming across the entire stack.


# Summarizing Using Prompt Templates

In [8]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [10]:
template = '''
Write a concise and short summary of the following text:
TEXT: `{text}`
Translate the summary to {language}.
'''
prompt = PromptTemplate(
    input_variables=['text', 'language'],
    template=template
)

In [11]:
llm.get_num_tokens(prompt.format(text=text, language='English'))

259

In [12]:
chain = LLMChain(llm=llm, prompt=prompt)
summary = chain.invoke({'text': text, 'language':'spanish'})

C:\Users\Vagner Machado\AppData\Local\Temp\ipykernel_7596\4218410605.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [16]:
print(summary)

{'text': 'Mojo es un nuevo lenguaje de programación que combina la usabilidad de Python con el rendimiento de C, desbloqueando una programabilidad sin igual de hardware de IA y la extensibilidad de modelos de IA. Con Mojo, puedes escribir código portátil más rápido que C y trabajar de forma integrada con el ecosistema de Python. Este lenguaje busca simplificar la programación en todo el stack de IA y ML, reconociendo la importancia de los aceleradores de hardware, incluyendo la CPU del host.', 'language': 'spanish'}


# Smmarizing with Stuff Document Chain
- Makes one single call to llm
- Model has access to whole text at once

In [17]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [18]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

docs = [Document(page_content=text)]
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [19]:
template = '''Write a concise and short summary of the following text.
TEXT: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Write a concise and short summary of the following text.\nTEXT: `{text}`\n')

In [22]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)
output_summary = chain.invoke(docs)

In [ ]:
# output_summary is a dict with 2 keys: 'input_documents' and 'output_text'
# displaying the summary
print(output_summary['output_text'])

## Summarizing Large Documents Using map_reduce

Say that the Model being used contains a limit for tokens. When that happens the map reduce approach will:
- Split the document into several chunks smaller than the model limit.
- Summarize each chunk
- Summarize all previous chunk summaries into one final summary
- Cons: It requires many calls to llm while there can also be some data / insight loss while summarizing summaries


In [26]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [27]:
with open('files/sj.txt', encoding='utf-8') as f:
    text = f.read()

docs = [Document(page_content=text)]
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [28]:
llm.get_num_tokens(text)

2653

In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50) # overlap from one chunk to other to provide continuity, set chunk size to 10k so jsut two calls are made
chunks = text_splitter.create_documents([text])

In [30]:
len(chunks)

# There are about 4 chars in a token

2

In [31]:
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
output_summary = chain.invoke(chunks)

In [32]:
# output_summary is a dict with 2 keys: 'input_documents' and 'output_text'
# displaying the summary
print(output_summary['output_text'])

Steve Jobs shares three stories from his life in his commencement speech, highlighting the importance of following one's passion, not settling for less, and living each day as if it were your last. He reflects on facing death, staying true to oneself, and embracing new opportunities. Jobs emphasizes the value of curiosity, idealism, and following one's own path in life.


In [33]:
# This is the prompt for summarizing each chunk

chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [35]:
# This is the prompt for combine all summaring and joining. 

chain.combine_document_chain.llm_chain.prompt.template

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

## map_reduce wich Custom Prompts

While above are predefined and work, maybe at times we need a custom prompt to e.g. summarize in bullet points. 

In [36]:
map_prompt = '''
Write a short and concise summary of the following:
Text: `{text}`
CONCISE SUMMARY:
'''
map_prompt_template = PromptTemplate(
    input_variables=['text'],
    template=map_prompt
)

In [37]:
combine_prompt = '''
Write a concise summary of the following text that covers the key points.
Add a title to the summary.
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
Text: `{text}`
'''
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=['text'])